# Secondo progetto di Social Computing

In [70]:
import pandas as pd

# importaimo il file .csv usando la libreria Pandas
df = pd.read_csv("group_5-Palma-Sacchet-Sagliocca.csv")

In [71]:
# %load utils.py
import json
import os


def serealize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder,exist_ok=True)
    with open(f"{folder}/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data,f,ensure_ascii=False,indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}.json")

def read_json(path):
    try:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as file:
                data = json.load(file)
            return data
    except ValueError:
        print("Path not found, check the correctness of the path")

In [72]:
# Funziona che permette di crere un elemento di un HIT (document)
def document_factory(row,human:bool):
    document = {
        'id': f'{row['id']}',
        'statement': row['statement'],
        'explanation': row['explanation_human'] if human else row['explanation_model'],
        'label': row['label']
    }
    return document

In [73]:
import random
from nanoid import generate
# Alfabeto utilizzato da nanodi per generare gli ID dei HIT
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Funzione per generare un HIT
def hit_factory(index:int):

    documents = []

    # L'obiettivo è selezionare d'apprima una coppia di righe tra le tre, queste avranno le spiegazioni generate
    model_df = df.sample(2)
    # E successivamente selezionare un altra riga sempre tra le tre, ques'ultima avrà la spiegazione umana
    human_df = df.sample()

    # Aggiungo i due elementi con explanation_model all'HIT
    for _, row in model_df.iterrows():
        document = document_factory(row,False)
        documents.append(document)
    # Aggiungo l'elemento con explanation_human all'HIT
    for _, row in human_df.iterrows():
        document = document_factory(row,True)
        documents.append(document)

    # Mescolo casualmente gli elementi dell'HIT
    random.shuffle(documents)

    # Creo l'HIT
    hit = {
        'unit_id': f'unit_{index}',
        'token_input': generate(alphabet,size=11), # Genera ID di 11 caratteri
        'token_output': generate(alphabet,size=11), # Genera ID di 11 caratteri
        'documents_number': len(documents),
        'documents': documents
    }
    
    return hit

In [74]:
# Inizializzo l'array dove mettero gli HITs
hits = []

# Genero i 12 HITs
for i in range(12):
    hit = hit_factory(i)
    hits.append(hit)

serealize_json(".","hits",hits)

Data serialized to path: ./hits.json


# Analisi dei dati

In [53]:
import pandas as pd
import re
import statistics

## Punto 10

In [57]:
# Carico il file csv da dove ricavo i dati
dataframe = pd.read_csv("csv/workers_notes.csv")

# Filtro solo le colonne che mi interessano
note_df = dataframe[['note_text_raw', 'note_text_left', 'note_text_right']]
# Converto le frasi in numeri (contando il numero di parole). Utilizzo regex per non considerare la punteggiatura
note_df_num = note_df.applymap(lambda x: len(re.findall(r'\w+', str(x))))
# Aggiungo una nuova colonna con il totale delle parole
note_df_num['note_text_total'] = note_df_num['note_text_raw'] + note_df_num['note_text_left'] + note_df_num['note_text_right']
# Aggiungo una nuova colonna con il valore percentuale
note_df_num['percentual'] = (note_df_num['note_text_raw'] / note_df_num['note_text_total']) * 100
# Calcolo la percentuale media
mean = statistics.mean(list(note_df_num['note_text_total']))
# Mostro il risultato
print(f"Percentuale media di testo annotato: {round(mean,2)}%")

Percentuale media di testo annotato: 33.05%
